# Project Task 2 - Song Recommendation (Part 1)

* Irem Ertürk - 03681130 - irem.erturk@tum.de
* Tulin Izer  - 03661686 - ga96pav@mytum.de

In [ ]:
import numpy as np
from itertools import count
from scipy import sparse

### Step 1: Get input from text file and create COO-matrix M

In [ ]:
n = 300000

row = []
col = []
data = []

user_to_row = {}
song_to_col = {}
user_count = count()
song_count = count()

with open('train_triplets.txt') as f:
    for _ in range(n):
        uid, sid, play_count = f.readline().split()
        
        if not uid in user_to_row:
            user_to_row[uid] = next(user_count)
        row.append(user_to_row[uid])

        if not sid in song_to_col:
            song_to_col[sid] = next(song_count)
        col.append(song_to_col[sid])        

        data.append(float(play_count))  #change as float because otherwise sparse matrix can not be used by svds

M = sparse.coo_matrix((data, (row, col)))

### Step 2: Preprocess the data by binning the play counts into b bins.

In [ ]:
b = 10

for item, data in enumerate(M.data):
    if (data>2**b):
        M.data[item] = b
    else :
        M.data[item] = len(bin(int(data)))-2#because casting the matrix as float I need to cast the data with int

### Step 3:Cold Start Issue

In [ ]:
def remove(A):
    csc = A.tocsc()
    cf = csc.indptr
    keepc = [i for i in range(len(cf)) if i < len(cf)-1 and cf[i+1] - cf[i] > 5]
    A = A[:, keepc]
    
    csr = A.tocsr() 
    rf = csr.indptr
    keepr = [i for i in range(len(rf)) if i < len(rf)-1 and rf[i+1] - rf[i] > 5]
    A = A[keepr, :]

    if len(keepc) == len(cf)-1 and len(keepr) == len(rf)-1:
        return A
    return remove(A)

In [ ]:
M = remove(M.tocsc())
M.shape

### Step 4:Test Data Selection


In [ ]:
indices = np.nonzero(M)

# Create 200 random indices for test data
rand = np.random.choice(len(indices[0]), 200, False)

test_data = np.zeros((200, 3), dtype=int)

for index, r in enumerate(rand):
    i = indices[0][r]
    j = indices[1][r]
    test_data[index] = [i, j, M[i,j]]
    M[i, j] = 0


### Step 5: Alternating Optimization

In [ ]:
# 5.1 : Find Q and P^T by SVD decomposition
from scipy.sparse.linalg import svds

init = 'svd'
k = 30 #factors
if init == 'svd':#!!Returns U,s and V^T actually and Initialize Q and P
    U, s, VT = svds(M, k=k)#Find single value decomposition U Σ and V^T(caution the last term!!)
    S = np.diag(s)#Return diagonal matrix format, small s returns vector format
    Q = U.dot(S)
    PT = VT


In [ ]:
Mc = M.tocsc()
Mr = M.tocsr()

def find_qis(c):
    qis_rowindex=[]
    col = Mc[:, c]
    nz = np.nonzero(col)
    ind = nz[0]  #row indeces
    #rxis = data[ind]
    rxis = np.array(col[nz])[0] 
    qis_rowindex = Q[ind, :]
    return qis_rowindex, rxis

def find_pxt(r):
    pxt_colindex=[]
    row = Mr[r, :]
    nz = np.nonzero(row)
    ind = nz[1]  #column indices
    #rxis = data[ind]
    rxis = np.array(row[nz])[0]
    ptx_columnindex = PT[:,ind]
    return ptx_columnindex, rxis

In [ ]:
from sklearn import linear_model
reg=linear_model.Ridge(fit_intercept=False)

def alternatingOptimization():
    for loop in range(10):
        #Optimize PT
        for x in range(PT.shape[1]):  #(PT.shape[1]):#loop through PT to optimize each ptx column-vector 
            qis, rxis=find_qis(x)
            if len(qis)>0 and len(rxis)> 0:
                reg.fit(qis,rxis)
                PT[:,x]=np.transpose(reg.coef_)
        #Optimize Q
        for i in range(Q.shape[0]):  #loop through Q to optimize each qi row-vector  
            pxt, rxis=find_pxt(i)       
            reg.fit(np.transpose(pxt),rxis)
            Q[i,:]=reg.coef_

In [ ]:
alternatingOptimization()

###  Step 6: Evaluate Model by Test Data

In [ ]:
from sklearn.metrics import mean_squared_error
predicted_values = np.zeros(200, dtype=float)
def evaluatemodel(test_data):
    for j in range(200):
        i = test_data[j,0]#row
        x = test_data[j,1]#column
        
        predicted_values[j] = Q[i,:].dot(PT[:,x])
        #print( Q[i,:], PT[:,x])
        RMSE=mean_squared_error(np.transpose(test_data[:,2]), predicted_values)
    return RMSE

In [ ]:
evaluatemodel(test_data) 